In [1]:
import numpy as np 
import pandas as pd

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
import heapq


C:\Users\User\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\User\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\User\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df_sample_submission = pd.read_csv('sample_submission.csv')
df_movies = pd.read_csv('movies.csv')
df_imdb = pd.read_csv('imdb_data.csv')
df_genome_scores = pd.read_csv('genome_scores.csv')
df_genome_tags = pd.read_csv('genome_tags.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_tags = pd.read_csv('tags.csv')
df_links = pd.read_csv('links.csv')

Since our original training data contains over 10 million rows, we will only train models on a subset of this. We will then choose the best performing model to be trained on the entire dataset. We subset our data by taking only those movies that have been reviewed over 500 times and only the users who have watched over 500 movies.

In [3]:
min_movie_ratings = 500
filter_movies = df_train['movieId'].value_counts() > min_movie_ratings
filter_movies = filter_movies[filter_movies].index.tolist()

min_user_ratings = 500
filter_users = df_train['userId'].value_counts() > min_user_ratings
filter_users = filter_users[filter_users].index.tolist()

df_new = df_train[(df_train['movieId'].isin(filter_movies)) & (df_train['userId'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df_train.shape))
print('The new data frame shape:\t{}'.format(df_new.shape))

The original data frame shape:	(10000038, 4)
The new data frame shape:	(750263, 4)


In [4]:
len(df_new)

750263

# Model-Based Collaborative Filtering

In this section we will focus on building a collaborative filtering recomendation system that uses machine learnin models to predict the ratings a user would assign to a movie.

The suprise package has been desinged for making recomendations using collaborative filtering and has a variety of built-in models.

### Data Prep

In [10]:
from surprise import Reader, Dataset
reader = Reader(rating_scale=(0.5, 5.0))

# Comparing base models

In [9]:
svd=SVD()

In [8]:
from surprise.model_selection import cross_validate

In [25]:
def Compare_models(data,models,subset):
    ratings = data[['userId','movieId','rating']][:subset]
    data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
    benchmark = []
    # Iterate over all algorithms
    for algorithm in models:
        # Perform cross validation
        results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)

        # Get results & append algorithm name
        tmp = pd.DataFrame.from_dict(results).mean(axis=0)
        tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
        benchmark.append(tmp)   
    return pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [26]:
models=[SVD(), SVDpp(), SlopeOne(), NMF(), KNNBasic(), KNNWithMeans(), CoClustering()]
compare_10K=Compare_models(df_new,models,10000)
compare_10K

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.965009,1.289254,0.065295
SVD,0.970174,0.454079,0.021987
KNNBasic,1.035287,0.087726,0.053046
SlopeOne,1.082335,0.198880,0.039639
KNNWithMeans,1.086155,0.106042,0.041647
NMF,1.170799,1.234707,0.034983
CoClustering,1.195125,0.927186,0.026110


In [27]:
models=[SVD(), SVDpp(), SlopeOne(), NMF(), KNNBasic(), KNNWithMeans(), CoClustering()]
compare_20K=Compare_models(df_new,models,20000)
compare_20K

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.927349,3.934849,0.183245
SVD,0.941286,1.085541,0.081285
KNNWithMeans,1.031659,0.094297,0.116429
KNNBasic,1.073730,0.073963,0.092945
SlopeOne,1.073845,0.292260,0.105594
NMF,1.078266,1.532294,0.043981
CoClustering,1.083353,0.947412,0.043973


In [28]:
models=[SVD(), SVDpp(), SlopeOne(), NMF(), KNNBasic(), KNNWithMeans(), CoClustering()]
compare_50K=Compare_models(df_new,models,50000)
compare_50K

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.878556,17.642252,0.727897
SVD,0.887191,3.185110,0.285398
CoClustering,0.972972,1.678862,0.116874
NMF,0.978862,3.632956,0.126934
KNNWithMeans,1.019516,0.145104,0.483214
KNNBasic,1.118027,0.125420,0.432924
SlopeOne,1.157686,0.325479,0.479392


As we can see, the SVD and SVDpp models perform the best when minimizing the rmse. These models have really high fit times which could cause problems when attempting to run on the entire dataframe.

In [31]:
(pd.DataFrame(compare_50K)['fit_time']+pd.DataFrame(compare_50K)['test_time'])*pd.DataFrame(compare_50K)['test_rmse']

Algorithm
SVDpp           16.139201
SVD              3.079003
CoClustering     1.747202
NMF              3.680413
KNNWithMeans     0.640581
KNNBasic         0.624245
SlopeOne         0.931788
dtype: float64

The KNN based approaches have the best performance when taking into account the time and the rmse, while the SlopeOne method also performs well using this metric. The SVD model does not do well on this metric, but the fit time is considerably lower than SVDpp while also having the best RMSE out of the rest of the models.

In [32]:
# Top 4 model
models=[SVD(), SlopeOne(),KNNBasic(),KNNWithMeans()]
compare_top=Compare_models(df_new,models,-1)
compare_top

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.791965,38.514202,2.729871
KNNWithMeans,0.810018,7.732693,81.783837
SlopeOne,0.818514,16.077581,108.419135
KNNBasic,0.844074,7.487565,77.077906


In [34]:
(pd.DataFrame(compare_top)['fit_time']+pd.DataFrame(compare_top)['test_time'])*pd.DataFrame(compare_top)['test_rmse']

Algorithm
SVD              32.663846
KNNWithMeans     72.509997
SlopeOne        101.902320
KNNBasic         71.379547
dtype: float64

When using over 750000 rows it seems that the SVD model has the edge in both rmse and total time.

# Making submission

In [35]:
def Make_prediction(model,data,test):
    data = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader).build_full_trainset()
    model.fit(data)
    prediction=[]
    for _,row in test.iterrows():
        prediction.append(model.predict(row[0],row[1]).est)
    Ident=[]
    for _,row in test.iterrows():
        Ident.append(str(row[0])+'_'+str(row[1])) 
    submission=pd.DataFrame([])
    submission['Id']=Ident
    submission['rating']=prediction
    return model,submission

In [ ]:
svd=SVD()
model,submission=Make_prediction(svd,df_train,df_test)

In [28]:
submission

,Id,rating
0,1_2011,3.136505
1,1_4144,4.367734
2,1_5767,3.539594
3,1_6711,4.217351
4,1_7318,2.876508
...,...,...
5000014,162541_4079,3.324737
5000015,162541_4467,3.971011
5000016,162541_4980,2.789832
5000017,162541_5689,2.951968


In [30]:
#submission.to_csv('SVD.csv', index = False)